In [1]:
import pandas as pd
import json
import os

In [9]:
# TODO: add the parser from https://github.com/ksreenivasan/gorilla/blob/pallavi_workspace/berkeley-function-call-leaderboard/kartik_analyze_fc_vs_prompt_generations.ipynb so that I can also view generations
def get_acc_df(model='databricks-dbrx-instruct-old-run'):
    # get all json files in score/ directory
    results_dir = f'score/{model}'
    json_files = [f'{results_dir}/{f}' for f in os.listdir(results_dir) if f.endswith('.json')]

    results_dict = {'filename': [], 'accuracy': [], 'correct_count': [], 'total_count': []}
    for filename in json_files:
        with open(filename, 'r') as f:
            data = [json.loads(line) for line in f.readlines()]
            accuracy_info = data[0]
            results_dict['filename'].append(filename)
            for key in accuracy_info.keys():
                results_dict[key].append(accuracy_info[key])

    results_df = pd.DataFrame(results_dict)
    results_df['metric'] = results_df['filename'].apply(lambda x: x.split('/')[-1].split('.')[0])

    overall_acc = results_df.correct_count.sum() / results_df.total_count.sum()
    print(f'Overall accuracy: {overall_acc}')
    return results_df

In [20]:
results_df = get_acc_df(model='databricks-dbrx-instruct-old-run')
results_df[['metric', 'accuracy', 'correct_count', 'total_count']]

Overall accuracy: 0.6676470588235294


,metric,accuracy,correct_count,total_count
0,javascript_score,0.520000,26,50
1,parallel_function_score,0.755000,151,200
2,executable_parallel_function_score,0.720000,36,50
3,executable_parallel_multiple_function_score,0.450000,18,40
4,parallel_multiple_function_score,0.500000,100,200
5,java_score,0.370000,37,100
6,relevance_score,0.483333,116,240
7,simple_score,0.827500,331,400
8,executable_multiple_function_score,0.760000,38,50
9,multiple_function_score,0.790000,158,200


In [18]:
fc_results_df = get_acc_df(model='databricks-dbrx-instruct-FC')
fc_results_df[['metric', 'accuracy', 'correct_count', 'total_count']]

Overall accuracy: 0.2644628099173554


,metric,accuracy,correct_count,total_count
0,simple_score,0.00,0,400
1,executable_parallel_function_score,0.00,0,50
2,rest_score,0.10,7,70
3,relevance_score,1.00,240,240
4,parallel_function_score,0.00,0,200
5,javascript_score,0.00,0,50
6,executable_simple_score,0.73,73,100
7,java_score,0.00,0,100


In [21]:
prompt_results_df = get_acc_df(model='databricks-dbrx-instruct')
prompt_results_df[['metric', 'accuracy', 'correct_count', 'total_count']]

Overall accuracy: 0.6471074380165289


,metric,accuracy,correct_count,total_count
0,parallel_function_score,0.735000,147,200
1,rest_score,0.614286,43,70
2,simple_score,0.832500,333,400
3,relevance_score,0.412500,99,240
4,javascript_score,0.460000,23,50
5,executable_parallel_function_score,0.680000,34,50
6,java_score,0.250000,25,100
7,executable_simple_score,0.790000,79,100


In [19]:
fc_results_df = get_acc_df(model='gpt-3.5-turbo-0125-FC')
fc_results_df[['metric', 'accuracy', 'correct_count', 'total_count']]

Overall accuracy: 0.5970588235294118


,metric,accuracy,correct_count,total_count
0,executable_parallel_function_score,0.740000,37,50
1,parallel_function_score,0.905000,181,200
2,simple_score,0.602500,241,400
3,java_score,0.540000,54,100
4,javascript_score,0.620000,31,50
5,executable_multiple_function_score,0.660000,33,50
6,multiple_function_score,0.695000,139,200
7,parallel_multiple_function_score,0.690000,138,200
8,executable_parallel_multiple_function_score,0.425000,17,40
9,relevance_score,0.016667,4,240


In [29]:
model_list = ['databricks-dbrx-instruct-FC', 'databricks-dbrx-instruct', 'gpt-3.5-turbo-0125-FC']
df1 = get_acc_df(model=model_list[0])
df2 = get_acc_df(model=model_list[1])

print(f'metric \t\t\t\t\t | {model_list[0]}_acc \t | {model_list[1]}_acc')
for metric in df1['metric'].unique():
    print(f'{metric} \t\t\t\t | {df1[df1["metric"] == metric]["accuracy"].item()} \t | {df2[df2["metric"] == metric]["accuracy"].item()}')

Overall accuracy: 0.2644628099173554
Overall accuracy: 0.6471074380165289
metric 					 | databricks-dbrx-instruct-FC_acc 	 | databricks-dbrx-instruct_acc
simple_score 				 | 0.0 	 | 0.8325
executable_parallel_function_score 				 | 0.0 	 | 0.68
rest_score 				 | 0.1 	 | 0.6142857142857143
relevance_score 				 | 1.0 	 | 0.4125
parallel_function_score 				 | 0.0 	 | 0.735
javascript_score 				 | 0.0 	 | 0.46
executable_simple_score 				 | 0.73 	 | 0.79
java_score 				 | 0.0 	 | 0.25
